<div class="alert alert-block alert-info">
    
# Ames Data Analysis


<div class="alert alert-block alert-warning">
    
# This notebook is just for illustrative purposes. The model fit here is *_not_* the best model for the data.

In [4]:
# This piece of code enables display of multiple output from one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [5]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Loading Data

In [6]:
ames = pd.read_csv("../Data/AmesHousing.csv", index_col="Order")
# Check dimensions of the data
print(" Dimensions of the imported dataset is: " + str(ames.shape))

 Dimensions of the imported dataset is: (2930, 81)


In [ ]:
# Print top 2 rows in the dataset
ames.head(4)

In [7]:
ames.columns = ames.columns.str.replace(' ', '')
ames.columns = [col.lower() for col in ames]
#ames.columns

In [8]:
# Marking the features or columns as numeric or categorical for easy referencce later in the code
numericfeatures = [f for f in ames.columns if ames.dtypes[f] != 'object']
print("Number of Numerical features: ", len(numericfeatures))

categoricalfeatures = [f for f in ames.columns if ames.dtypes[f] == 'object']
print("Number of Categorical features: ", len(categoricalfeatures))

Number of Numerical features:  38
Number of Categorical features:  43


In [ ]:
print("numeric features: " + str(numericfeatures))
print("categorical features: " + str(categoricalfeatures))

In [ ]:
# What is the distribution of sale price
ames['saleprice'].hist(bins=75, rwidth=.8, figsize=(14,4))
ames['saleprice'].describe()

The minimum sale price is at 12,789 while the maximum sale price is 755,000. The median sale price was $160,000. The distribution seems to be skewed with a few very high priced house sales on the right

In [ ]:
# Date range in the data
yax = ames.groupby(['yrsold'])['yrsold'].count()
plt.bar(yax.index, yax)

The data was collected from 2006 to 2010. 
Note: Data includes the 2008 downturn event. Effects of which in the data is yet to be seen

In [ ]:
from datetime import date
date_sold = pd.DataFrame({'year': ames['yrsold'], 'month' : ames['mosold']})

DATE = []
for yr, mo in zip(date_sold.year, date_sold.month):
    DATE.append(date(yr, mo, 10))

ames['yrmosold'] = DATE
ames.head()

In [ ]:
# Is there seasonality in house sales
ames.groupby(['yrmosold']).pid.count().plot(kind='line', figsize=(14,4))
plt.title('Seasonality in Houses sold')
plt.ylabel('# of houses sold')
plt.xlabel('Sale Year-Month')

There is a clear seasonal pattern in house sales, peaking in summer. However 2010 seems to be different from the previous few years in number of houses sold.

In [ ]:
ames.groupby(['neighborhood']).pid.count().sort_values(ascending=False).plot(kind='bar', figsize=(14,4))
plt.title('Number of houses sold by neighborhood')
plt.ylabel('# of houses sold')

ames[['saleprice','neighborhood']].boxplot(column=['saleprice'], by=['neighborhood']
                                           ,rot=45, fontsize=15, figsize=(20,15))

NAmes had the highest number of house sold in the sample data. NWAmes had a few houses sold at the highest price in the dataset.

## Clean Data

1. Drop duplicates
2. Handle missing values

#### Check and Drop Duplicates

In [ ]:
# Drop duplicates
before = ames.shape[0]
ames.drop_duplicates(inplace = True, keep = 'last')
rows_dropped = before - ames.shape[0]
print(str(rows_dropped) + ' ' +  "duplicate rows dropped from the the dataset")

#### Check and Handle Missing Values

In [ ]:
#Check for missing values
print("Columns by % of missing values in descending order")
ames.isnull().sum()[ames.isnull().sum()>0].sort_values(ascending = False)*100/ames.shape[0]

In [ ]:
# Since most of the categorical data that is missing is due to absence of feature in the home. 
# So we are imputing the missing values with "none" 
# Impute missing values


for col in categoricalfeatures:
    ames[col].fillna('none',inplace=True)

In [ ]:
# To illustrate, we are imputing the missing values for numeric features with median.
ames.fillna(ames.median(), inplace=True)

In [ ]:
#Check for missing values
print("Columns by % of missing values in descending order")
ames.isnull().sum()[ames.isnull().sum()>0].sort_values(ascending = False)*100/ames.shape[0]

## EDA

### Univariate

In [ ]:
# plot histogram chart
ames[numericfeatures].hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)
    

Some of the numeric features seem to be categories. 
There are features that indicate time 

In [ ]:
for col in list(categoricalfeatures) :
    print(ames[col].value_counts())
    print('#'*50)

In [ ]:
# Drop categorical features that are highly skewed 
before = ames.shape[1]
drop_cols =  ['heating', 'roofmatl','condition2', 'street', 'utilities', 'alley']
ames.drop(drop_cols, axis = 1, inplace = True)
columns_dropped = before - ames.shape[1]
print(str(columns_dropped) + ' ' + 'columns dropped from the original dataset')

## Feature Engineering 
1. Create categorical features
2. Take math transformation

In [ ]:
ames['log_saleprice'] = np.log(ames['saleprice'])

In [ ]:
ames['age']= ames['yrsold']-ames['yearbuilt']

In [ ]:
ames['totalbath'] = ames['fullbath'] + ames['halfbath']*0.5 + ames['bsmtfullbath'] + ames['bsmthalfbath']*0.5
ames['totalsqft']= ames['totalbsmtsf'] + ames['1stflrsf'] + ames['2ndflrsf']

In [ ]:
neighmap  = {'NoRidge':3,'NridgHt':3, 'Somerst':3,
             'NAmes':1,'Sawyer':1,'OldTown':1,'Edwards':1,'BrkSide':1, 
             'CollgCr':2, 'NWAmes': 2, 'SawyerW':2, 'Mitchel':2, 'Crawfor': 2, 'IDOTRR': 2, 'Timber': 2,
             'StoneBr': 2, 'SWISU': 2, 'ClearCr': 2, 'MeadowV': 2, 'BrDale': 2, 'Blmngtn': 2, 'Veenker':2,
             'NPkVill': 2, 'Blueste': 2, 'Greens': 2, 'GrnHill': 2, 'Landmrk': 2, 'Crawfor': 2, 'Gilbert':2}
ames['neighborbuckets'] = ames['neighborhood'].map(neighmap)

In [ ]:
ames.head(3)

In [ ]:
# created boolean features
ames['hasbasement'] = ames.totalbsmtsf.apply(lambda x: 1 if x > 0 else 0)
ames['hasgarage'] = ames.garagearea.apply(lambda x: 1 if x > 0 else 0)
ames['haspool'] = ames.poolarea.apply(lambda x: 1 if x > 0 else 0)
ames['wasremodeled'] = (ames['yearremod/add'] != ames.yearbuilt).astype(np.int64)

In [ ]:
# Marking the features or columns as numeric or categorical for easy referencce later in the code
numericfeatures = [f for f in ames.columns if ames.dtypes[f] != 'object']
print("Number of Numerical features: ", len(numericfeatures))

categoricalfeatures = [f for f in ames.columns if ames.dtypes[f] == 'object']
print("Number of Categorical features: ", len(categoricalfeatures))

## Visualization

### Univariate

In [ ]:
ames[numericfeatures].hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
for col in list(categoricalfeatures) :
    print(ames[col].value_counts())
    print('#'*50)

### Bivariate

In [ ]:
ames_corr = ames.corr()
ames_corr['saleprice'].sort_values(ascending=False)
#ames['log_saleprice'] = np.log(ames['saleprice'])

In [ ]:
subsetd = ames[['saleprice','log_saleprice','grlivarea','totalsqft','totalbath','hasbasement',
                    'overallqual', 'hasgarage','neighborbuckets','age']]
#pd.plotting.scatter_matrix(subsetd, alpha=0.9, marker="o", figsize=(15,15),diagonal = 'kde', grid=True)

In [ ]:
subsetd['saleprice'].hist(bins=50, rwidth=.8, figsize=(14,4))
plt.show()

subsetd['log_saleprice'].hist(bins=50, rwidth=.8, figsize=(14,4))
plt.show()

In [ ]:
subsetd = ames[['log_saleprice','grlivarea','totalsqft','totalbath','hasbasement',
                    'overallqual', 'hasgarage','neighborbuckets','age']]
#pd.plotting.scatter_matrix(subsetd, alpha=0.9, marker="o", figsize=(15,15),diagonal = 'kde', grid=True)

There seems to be positive correlations with log_sale price.

## Is there any difference between the neighborhood buckets?

In [ ]:
ames[['saleprice','neighborbuckets']].boxplot(column=['saleprice'], by=['neighborbuckets']
                                           ,rot=45, fontsize=15, figsize=(20,15))

In [ ]:
from scipy.stats import f_oneway
n1 = ames[ames.neighborbuckets == 1]
n2 = ames[ames.neighborbuckets == 2]
n3 = ames[ames.neighborbuckets == 3]
stat, p = f_oneway(n1['saleprice'], n2['saleprice'], n3['saleprice'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Mean of all the categories are the same')
else:
    print('At least one mean is different')

## Verify Sale price distribution

In [ ]:
stats.probplot(ames['saleprice'], plot=plt)
plt.ylabel('Sale Price')
plt.show()
stats.probplot(np.log(ames['saleprice']), plot=plt)
plt.ylabel('Log Sale Price')
plt.show()

Eyeballing the normal probability plot for Sale price seems like it has fatter tails as compared to the normal distribution. Taking log transformation of the sale price seems to pass a fat pencil test.
Ho: Sale Price is normal (or Guassian) distributed
H1: Sale price is not normal distributed

## Hypothesis Testing

In [ ]:
subsetd = ames[['log_saleprice','log_grlivarea','totalsqft','totalbath','hasbasement',
                    'overallqual', 'hasgarage','neighborbuckets','age']]


In [ ]:
from scipy.stats import shapiro
series_2test = ames[['saleprice']]
stat, p = shapiro(series_2test)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

In [ ]:
ames['log_grlivarea'] = np.log(ames['grlivarea'])

In [ ]:
ames['grlivarea'].hist(bins=50, rwidth=.8, figsize=(14,4))
plt.show()
ames['grlivarea'].describe()

stats.probplot(ames['grlivarea'], plot=plt)
plt.ylabel('grlivarea')
plt.show()
stats.probplot(np.log(ames['grlivarea']), plot=plt)
plt.ylabel('log Gr Liv Area')
plt.show()

## Predictive Modeling

In [ ]:
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split

target= subsetd['log_saleprice']
df_train = subsetd.drop(['log_saleprice'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(df_train,target,test_size=0.33,random_state=0)
# Split data into train and test
ames_train = pd.concat([x_train, y_train], axis=1)
#ames_train.head()
ames_test = pd.concat([x_test, y_test], axis=1)

In [ ]:
ames_train.shape
ames_test.shape

In [ ]:
formula = 'log_saleprice ~ totalsqft + totalbath + overallqual \
            + C(neighborbuckets) + age'
ames_model = ols(formula, data=ames_train).fit()
print(ames_model.summary())

In [ ]:
print('Parameters: ', ames_model.params)
print('R2: ', ames_model.rsquared)

print('Standard errors: ', ames_model.bse)
print('Predicted values: ', ames_model.predict())

## Ref:https://www.statsmodels.org/stable/examples/notebooks/generated/ols.html

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(x_train,y_train)
print(lin_reg.fit)
print(lin_reg.intercept_)
coeff_df = pd.DataFrame(lin_reg.coef_, x_train.columns, columns=['Coefficient'])
coeff_df

In [ ]:
# Check for multicollinearity

In [ ]:
subsetd.corr().style.background_gradient(cmap='coolwarm').set_precision(2)

### Time Series 

In [ ]:
ames_ts = pd.DataFrame(ames.groupby(['yrmosold']).pid.count())
ames_ts.info()
ames_ts.head()
ames_ts.tail()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ames_ts.pid.plot(ax=ax);
legend = ax.legend(loc = 'upper left');
legend.prop.set_size(20);

## Decomposition of Time series into seasonal and trend 

In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(pd.Series(ames_ts.pid),period =12)
res = stl.fit()
fig = res.plot()

### Extra

### Plotting ACF and PACF

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA


fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(ames_ts.pid.squeeze(), lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(ames_ts, lags=24, ax=ax2)

## Figuring whether Multiplicative model or aditive model is more useful

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

# Multiplicative Decomposition 
result_mul = seasonal_decompose(ames_ts, model='multiplicative', period=12)

# Additive Decomposition
result_add = seasonal_decompose(ames_ts, model='additive', period=12)

# Plot
plt.rcParams.update({'figure.figsize': (10,10)})
result_mul.plot().suptitle('Multiplicative Decompose', fontsize=22)
result_add.plot().suptitle('Additive Decompose', fontsize=22)
plt.show()

## Fitting Models

In [ ]:
arma_mod = ARIMA(ames_ts, order=(1, 2, 0)).fit()
#print(arma_mod.summary())

In [ ]:
arma_mod2 = ARIMA(ames_ts, order=(1, 0, 0)).fit()
#print(arma_mod2.params)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax = arma_mod.resid.plot(ax=ax);
ax = fig.add_subplot(111)
ax = arma_mod2.resid.plot(ax=ax);